# QC4QC 2024 tutorial: Noise in Quantum Computing

For this tutorial, we need features that are not in myQLM, but are present in Qaptiva (the full product).
We have access to a distant Qaptiva, to which we need first to connect. Because everyone is going to use it at the same time, **limit your circuits to no more than 5 qubits!**
Enter username and password given by Eviden staff.

In [ ]:
from qat.qlmaas import QLMaaSConnection

conn = QLMaaSConnection(hostname="qlm35e.neasqc.eu")
conn.create_config() 

## Noisy QFT: a first example

We first study a simple circuit: a Quantum Fourier Transform (QFT) on 3 qubits. It only requires Hadamard and controlled-phase gates. This code produce this circuit:

In [ ]:
from qat.lang.AQASM import Program
from qat.lang.AQASM.qftarith import QFT
nqbits = 3
prog = Program()
reg = prog.qalloc(nqbits)
prog.apply(QFT(nqbits), reg)
qft_circuit = prog.to_circ(inline=True) #convert program to circuit
qft_circuit.display()

We emulate this circuit on the distant Qaptiva, without noise:

In [ ]:
# from qat.qpus import get_default_qpu
from qlmaas.qpus import LinAlg

job = qft_circuit.to_job('SAMPLE')
qpu = LinAlg()
res = qpu.submit(job).join()

res.plot()

We want to know what is the outcome of this circuit when qubits are noisy. We assume they are submitted to decoherence when no gate is applied on them (idle qubit). We model the decoherence phenomenon by two quantum channels:

* Amplitude damping ($T_1$)

* Pure Dephasing ($T_{\phi}$)

### Gate specifications

We need to tell Qaptiva how long each gate takes to apply, so that it can deduce when and how long qubits are idle. This is done in a `GateSpecification` object, which contains gate time information but also what operation each gate does.

The `DefaultGateSpecification` allows to produce a `GateSpecification` with all the standard gate operations and some default gate time. Here we use it and overwrite the gate times with our own:

In [ ]:
gate_times = {"H":60, "C-PH":lambda angle :150}  # in nanoseconds

from qat.hardware import DefaultGatesSpecification
gates_spec = DefaultGatesSpecification(gate_times)

### Noise quantum channels

Now let's choose the effect of the noise. Qaptiva has already defined an amplitude damping and a pure dephasing channel:

In [ ]:
from qat.quops import ParametricPureDephasing, ParametricAmplitudeDamping
T1, T2 = 4400, 3890  # in nanoseconds

ad_channel = ParametricAmplitudeDamping(T_1 = T1)

pd_channel = ParametricPureDephasing(T_phi = 1/(1/T2 - 1/(2*T1)))

### Hardware model = Gate Specification + quantum channels

Now we combine both information in a single `HardwareModel` object:

In [ ]:
from qat.hardware import HardwareModel

hardware_model = HardwareModel(gates_spec, idle_noise=[ad_channel, pd_channel])


### Noisy emulation

Time to get the result! In Qaptiva, everything about noise or the imperfections of the quantum computer is contained in a `HardwareModel` object and given as a parameter of a QPU emulator.
Only some QPU can handle noise, and the first choice should be `NoisyQProc`, which performs exact emulation.

In [ ]:
from qlmaas.qpus import NoisyQProc

job = qft_circuit.to_job('SAMPLE')
noisy_qpu = NoisyQProc(hardware_model)
res = noisy_qpu.submit(job).join()

res.plot()

## Noisy VQE: adapt to your previous tutorials circuits

Now it is your turn! Use what you learned to emulate the variational algorithm of this morning's tutorial, but with noisy qubits.

In the next few cells, we reproduced the code necessary for the VQE calculation you did previously. Adapt it to take noise into account in the simulation.

Hint: you need to give the time of all gates used in your circuit!

Observe the result and compare with noiseless emulation. Does VQE perform better/worse? How does it depend on $T_1$ and $T_2$? On gate times? On the number of layers?


In [ ]:
## Fill in the gaps "......" and copy your circuit from the previous tutorial

gate_times = {"X":60, "CNOT": 120, "RY": lambda theta: 60}  # in nanoseconds
gates_spec = DefaultGatesSpecification(gate_times)

T1, T2 = 4400, 3890  # in nanoseconds
ad_channel = ParametricAmplitudeDamping(T_1 = T1)
pd_channel = ParametricPureDephasing(T_phi = 1/(1/T2 - 1/(2*T1)))

hardware_model = HardwareModel(gates_spec, idle_noise=[ad_channel, pd_channel])


In [ ]:
import numpy as np
from qat.plugins import ScipyMinimizePlugin
from qat.lang.AQASM import Program, X, RX, RY, RZ, CNOT
from qat.fermion.chemistry import MolecularHamiltonian
from qat.fermion.transforms import transform_to_jw_basis

h2_data = np.load("data/h2_data.npz", allow_pickle=True)

rdm1 = h2_data["rdm1"]
nuclear_repulsion = h2_data["nuclear_repulsion"]
one_body_integrals = h2_data["one_body_integrals"]
two_body_integrals = h2_data["two_body_integrals"]
noons, basis_change = np.linalg.eigh(rdm1)
nqbits = rdm1.shape[0] * 2  # Assuming each orbital corresponds to 2 qubits (spin-up and spin-down)

mol_h = MolecularHamiltonian(one_body_integrals, two_body_integrals, nuclear_repulsion)
Ham = mol_h.get_electronic_hamiltonian()
H_sp = transform_to_jw_basis(Ham)

In [ ]:

prog = Program()
reg = prog.qalloc(H_sp.nbqbits)

for i in range(2):
    prog.apply(X, reg[i])

theta_list = [prog.new_var(float, "\\theta_{%s}" % i) for i in range(32)]

for l in range(4):
    for i in range(4):
        prog.apply(RY(theta_list[i + 4 * l]), reg[i])
    for i in range(3):
        prog.apply(CNOT, reg[i], reg[i + 1])
for i in range(4):
    prog.apply(RY(theta_list[i + 4 * (l + 1)]), reg[i])

circ = prog.to_circ()
circ.display()


In [ ]:
## Plugins should be imported from qlmaas:
from qlmaas.plugins import ScipyMinimizePlugin, ObservableSplitter

job = circ.to_job(observable=H_sp, nbshots=0)
optimizer_scipy = ScipyMinimizePlugin(method="COBYLA", tol=1e-3, options={"maxiter": 1000})
qpu = optimizer_scipy | ObservableSplitter() | NoisyQProc(hardware_model)
## The ObservableSplitter split the hamiltonian in terms that can actually be measured on an actual quantum computer

result = qpu.submit(job)

print("Minimum energy =", result.value)


## More advances Hardware Models

So far, we only saw a basic noise model. With Qaptiva, you can further:
* apply different noise on different qubits
* create your own quanutm channels, for personalized coupling to the environment
* emulate State Preparation And Measurement (SPAM) errors
* emulate faulty gates
* combine all that!

Let's try having different $T_1$ and $T_2$ for different qubits (fill in the blanks):

In [ ]:
nb_qubits = ...

T1_list = [4400, 3500, ......]  # in nanoseconds
T2_list = [3890, 3950, ......]  # in nanoseconds
ad_channel_list = [ParametricAmplitudeDamping(T_1 = T1) for T_1 in T_1_list]
pd_channel_list = [ParametricPureDephasing(T_phi = 1/(1/T2 - 1/(2*T1))) for T_1, T_2 in zip(T_1_list, T_2_list)]

idle_noise = {qubit_index: [ad_channel_list[qubit_index], pd_channel_list[qubit_index]] for qubit_index in range(nb_qubits)}

hardware_model = ......
